# Introduction the pairinteraction library

## Datatypes
The pairinteraction software supports real and complex datatypes in its C++ backend implementation to adapt to the specific needs of the user.
The `real` datatype ise used for real-valued calculations (typically if no field in y-direction is present), while the `complex` datatype is used for complex-valued calculations, i.e. when using external fields in y-direction.
To change the datatype it is as simple as changing this import statement:

In [1]:
import numpy as np

import pairinteraction.real as pi  # possible backend data types: real, complex

np.set_printoptions(linewidth=120, threshold=10)

## The Database object
The `Database` object is responsible for storing and looking up the allowed atomic states with their corresponding energies, quantum numbers and electric dipole, etc. matrix elements with other atomic states.
These matrix elements are pre-calculated (either via explicit calculation of the overlap integrals and using the Numerov method to get the radial wavefunctions, or alternatively for Earth Alkali atoms via Multichannel Quantum Defect Theory (MQDT)) and stored online in their own github repositories.
The `Database` object is able to download the necessary tables on the fly if `download_missing=True` is passed to the `Database`. Once downloaded, the tables are stored in the cache directory of the user's computer and are reused in subsequent calculations.

You can either create a `Database` object via `database = pi.Database(download_missing=True)` and use this database for the creation of the kets and basis objects below,
or alternatively you can once create a global instance of the `Database` object via `pi.Database.initialize_global_database(download_missing=True)` and then the ket and basis classes will use this global instance by default.

In [2]:
if pi.Database.get_global_database() is None:
    pi.Database.initialize_global_database(download_missing=True)

# KetAtom
The simplest object you can create is a simple ket, e.g. via the `KetAtom` class.
The `KetAtom` object represents a single atomic state.
The first argument has to be the specifier of the atomic species.
Currently supported species are: ('Rb', 'Sr88_singlet', 'Sr88_triplet', 'Sr87_mqdt', 'Sr88_mqdt', 'Yb171_mqdt', 'Yb174_mqdt'), where no ending or '_singlet' or '_triplet' specifies that the matrix elements are calculated via the Numerov method, while '_mqdt' specifies that the matrix elements are calculated via MQDT.
The following optional keyword arguments of KetAtom have to uniquely specify the atomic state, you can pass whatever combination of quantum numbers you like, as long as they uniquely specify exactly one state (e.g. `pi.KetAtom("Rb", n=60, l=0, j=0.5, m=0.5)` and `pi.KetAtom("Rb", n=60, l=0, m=0.5)` are both equivalent and specify the same state).

In [3]:
ket = pi.KetAtom("Rb", n=60, l=1, j=0.5, m=0.5)

print(
    f"You successfully created a ket object: {ket} "
    f"with quantum numbers n={ket.n}, l={ket.l}, j={ket.j}, m={ket.m} and energy={ket.energy}"
)
print(f"And getting its energy in GHz is as simple as {ket.get_energy(unit='GHz')=}")

ket2 = pi.KetAtom("Rb", n=58, l=0, j=0.5, m=0.5)
delta_energy = ket.energy - ket2.energy  # this is a pint.Quantity object

print("Even the energy difference between two states is easily calculated and converted:")
print(f"delta_energy = {delta_energy}")
print(13 * " " + f"= {delta_energy.to('GHz', 'spectroscopy')} (as frequency)")
print(13 * " " + f"= {delta_energy.to('cm^-1', 'spectroscopy')} (as wavenumber)")
print(13 * " " + f"= {delta_energy.to('eV', 'spectroscopy')} ")
print(13 * " " + f"= {delta_energy.to('mm', 'spectroscopy')} (as wavelength)")

You successfully created a ket object: |Rb:60,P_1/2,1/2⟩ with quantum numbers n=60, l=1.0, j=0.5, m=0.5 and energy=0.15335520073991077 bohr ** 2 * electron_mass / atomic_unit_of_time ** 2
And getting its energy in GHz is as simple as ket.get_energy(unit='GHz')=1009028.7484337102
Even the energy difference between two states is easily calculated and converted:
delta_energy = 1.403362628490945e-05 bohr ** 2 * electron_mass / atomic_unit_of_time ** 2
             = 92.33682521314954 gigahertz (as frequency)
             = 3.0800249555694146 / centimeter (as wavenumber)
             = 0.0003818744252705271 electron_volt 
             = 3.2467269402858667 millimeter (as wavelength)


# BasisAtom
Next, you can create a basis object.
A basis object consists of a list of kets, which define a canonical basis for the Hilbert space.
Furthermore, the basis object defines basis states via its coefficients matrix, where each column in the coefficients matrix corresponds to one basis state.
When created the coefficients matrix is initialized to the identity matrix, i.e. each basis state correspond to one ket.
However, in general a state (and therefore each column of the basis coefficients matrix) can be a superposition of multiple kets.

The list of which kets should be considered in the basis can be restricted by passing in tuples of (min, max) values for the quantum numbers and the energy to the `BasisAtom` class.

In [4]:
energy_min, energy_max = ket.get_energy(unit="GHz") - 100, ket.get_energy(unit="GHz") + 100
basis = pi.BasisAtom("Rb", n=(58, 63), l=(0, 3), energy=(energy_min, energy_max), energy_unit="GHz")
coefficients = basis.coefficients  # this is a scipy.sparse.csr_matrix object

print(f"This basis contains {basis.number_of_kets} kets (=atomic states)")
print(f"The first and last kets are {basis.kets[0]} and {basis.kets[-1]}")
print()
print(f"The coefficient matrix has shape {coefficients.shape} and the following entries:")
print(f"{coefficients.toarray()}")

This basis contains 130 kets (=atomic states)
The first and last kets are |Rb:58,S_1/2,-1/2⟩ and |Rb:63,P_3/2,3/2⟩

The coefficient matrix has shape (130, 130) and the following entries:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


# SystemAtom
The `SystemAtom` object describes the single atom system.
It is created by passing a `BasisAtom` object in, which defines the basis of the Hilbert space.
You can now set external fields and enable or disable the diamagnetic term.

Then you can inspect the resulting Hamiltonian and diagonalize it to get the eigenstates and eigenenergies of the system.

In [5]:
system = pi.SystemAtom(basis)
system.set_magnetic_field([0, 0, 1], unit="gauss")
system.set_electric_field([0.1, 0, 0.1], unit="V/cm")
system.enable_diamagnetism(True)

print("The Hamiltonian of the system (in GHz) with magnetic and electric fields is:")
print(f"{system.get_hamiltonian(unit='GHz').toarray()}")

The Hamiltonian of the system (in GHz) with magnetic and electric fields is:
[[ 1.00893641e+06  0.00000000e+00  1.48688631e-01 ...  2.75971999e-03 -1.37985999e-03  0.00000000e+00]
 [ 0.00000000e+00  1.00893641e+06 -1.48688631e-01 ...  1.37985999e-03  2.75971999e-03 -2.38998762e-03]
 [ 1.48688631e-01 -1.48688631e-01  1.00895514e+06 ... -1.73384434e-06  0.00000000e+00  0.00000000e+00]
 ...
 [ 2.75971999e-03  1.37985999e-03 -1.73384434e-06 ...  1.00912614e+06  0.00000000e+00  0.00000000e+00]
 [-1.37985999e-03  2.75971999e-03  0.00000000e+00 ...  0.00000000e+00  1.00912614e+06  0.00000000e+00]
 [ 0.00000000e+00 -2.38998762e-03  0.00000000e+00 ...  0.00000000e+00  0.00000000e+00  1.00912614e+06]]


In [6]:
system.diagonalize()
eigenvalues = system.get_eigenvalues(unit="GHz")
print("The eigenenergies in GHz are:")
print(eigenvalues)

The eigenenergies in GHz are:
[1008936.40335628 1008936.40615876 1008955.13771731 ... 1009126.14540512 1009126.15112008 1009126.15407509]


In [7]:
eigenbasis = system.get_eigenbasis()
ket = pi.KetAtom("Rb", n=60, l=1, j=0.5, m=0.5)
corresponding_state = eigenbasis.get_corresponding_state(ket)

eigenstate_number = 2
print(f"The eigenvector with index {eigenstate_number} is:")
print(f"{eigenbasis.coefficients.toarray()[:, eigenstate_number]}")
print(f"It has the largest overlap with the ket: {eigenbasis.get_corresponding_ket(eigenstate_number)}")


print(f"The state corresponding to the ket {ket} is:")
print(f"{corresponding_state.coefficients.toarray().flatten()}")
print(f"The overlap |<state|ket>|^2 is {corresponding_state.get_overlaps(ket)[0]}")

The eigenvector with index 2 is:
[ 0.00793871 -0.00792766  0.99988698 ...  0.          0.          0.        ]
It has the largest overlap with the ket: |Rb:58,P_1/2,-1/2⟩
The state corresponding to the ket |Rb:60,P_1/2,1/2⟩ is:
[-7.62549565e-05 -7.64093445e-05  0.00000000e+00 ...  4.18012523e-06  1.41482889e-06 -2.41644012e-06]
The overlap |<state|ket>|^2 is 0.9982977951678482


# BasisPair
The `BasisPair` object consists of a list of `KetPair` objects.
Again, we view these KetPair kets as forming a canonical basis for the pair Hilbert space.
However, in contrast to the `KetAtom` objects, the `KetPair` objects are not atomic states (or product states of atomic states), but rather product states of the eigenstates of the single atom Hamiltonian.

Again, the `BasisPair` object has a coefficients matrix, which defines the basis states (with respect to the list of `KetPair` objects).
The coefficients matrix is initialized to the identity matrix.
This corresponds to the eigenstates of the pair Hamiltonian with external fields but without any interaction between the atoms.
In general, when adding interactions, the pair-states (=the columns of the coefficent matrix) can be a superposition of multiple KetPair objects.

Similar to the `BasisAtom` object, we can restrict the list of kets that should be considered in the basis by passing in tuples of (min, max) values for the energy (of the pair states) and the quantum number m if it is conserved.


In [8]:
pair_energy = 2 * system.get_corresponding_energy(ket, unit="GHz")
delta_energy = 3  # GHz
pair_basis = pi.BasisPair(
    [system, system],
    energy=(pair_energy - delta_energy, pair_energy + delta_energy),
    energy_unit="GHz",
)
coefficients = pair_basis.coefficients

print(f"The pair-basis contains {pair_basis.number_of_kets} KetPair (=pair-states)")
print(f"The first KetPair corresponds to a pair-state close to the product state {pair_basis.kets[0]}")
print()
print(f"The coefficient matrix has shape {coefficients.shape} and the following entries:")
print(f"{coefficients.toarray()}")

The pair-basis contains 580 KetPair (=pair-states)
The first KetPair corresponds to a pair-state close to the product state |Rb:58,P_1/2,-1/2; Rb:61,D_3/2,-3/2⟩

The coefficient matrix has shape (580, 580) and the following entries:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


# SystemPair
The `SystemPair` object describes the pair system.
Similar to the `SystemAtom` object, it is created by passing a `BasisPair` object in, which defines the basis of the pair Hilbert space.
You can now set the interatomic distance between the atoms and the order of the multipole expansion.
Then you can inspect the resulting Hamiltonian and diagonalize it to get the eigenstates and eigenenergies of the system.

In [9]:
pair_system = pi.SystemPair(pair_basis)
distance = 5  # micrometer
pair_system.set_distance(distance, unit="micrometer")
pair_system.set_order(3)

print(f"The Hamiltonian of the SystemPair (in GHz) for the given {distance=}um is:")
print(f"{pair_system.get_hamiltonian(unit='GHz').toarray()}")

The Hamiltonian of the SystemPair (in GHz) for the given distance=5um is:
[[ 2.01805976e+06  6.30194544e-09  1.35466497e-06 ... -3.29215113e-10  5.56627090e-10 -2.42208422e-10]
 [ 6.30194544e-09  2.01805977e+06 -1.02994569e-06 ... -1.31926787e-09 -5.63476847e-10  1.75820162e-09]
 [ 1.35466497e-06 -1.02994569e-06  2.01805977e+06 ... -7.42015942e-10 -3.19784432e-10 -1.82370533e-10]
 ...
 [-3.29215113e-10 -1.31926787e-09 -7.42015942e-10 ...  2.01806034e+06 -1.85611056e-08 -1.39796132e-07]
 [ 5.56627090e-10 -5.63476847e-10 -3.19784432e-10 ... -1.85611056e-08  2.01806034e+06 -5.26445479e-08]
 [-2.42208422e-10  1.75820162e-09 -1.82370533e-10 ... -1.39796132e-07 -5.26445479e-08  2.01806034e+06]]


In [10]:
pair_system.diagonalize()
eigenvalues = pair_system.get_eigenvalues(unit="GHz")
print("The eigenenergies in GHz are:")
print(eigenvalues)

The eigenenergies in GHz are:
[2018054.77997889 2018054.77998195 2018054.78225167 ... 2018060.3415922  2018060.34301859 2018060.34301872]


# Calculate electric dipole matrix elements

In [ ]:
ket1 = pi.KetAtom("Rb", n=60, l=0, j=0.5, m=0.5)
ket2 = pi.KetAtom("Rb", n=60, l=1, j=0.5, m=0.5)


database = pi.Database.get_global_database()
electric_dipole = ket1.get_matrix_element(ket2, operator="electric_dipole", q=0)
magnetic_dipole = ket1.get_matrix_element(ket1, operator="magnetic_dipole", q=0)

print(f"The electric dipole matrix element between {ket1} and {ket2} is {electric_dipole}")
print(f"The magnetic dipole matrix element between {ket2} and itself is {magnetic_dipole}")

The electric dipole matrix element between |Rb:60,S_1/2,1/2⟩ and |Rb:60,P_1/2,1/2⟩ is 1247.5985544327225 atomic_unit_of_current * atomic_unit_of_time * bohr
The magnetic dipole matrix element between |Rb:60,P_1/2,1/2⟩ and itself is -0.5005798 atomic_unit_of_current * bohr ** 2
